# OpenMKM Input and Output
This notebook describes pmutt's functionality to read and write OpenMKM CTI files. We will use the NH3 formation mechanism as a case study.

## Topics Covered
- Read species *ab-initio* data, reactions, lateral interactions and phases from a spreadsheet
- Write the CTI file that can be read by OpenMKM

## Input Spreadsheet
All the data will be imported from the [`./inputs/NH3_Input_data.xlsx`](https://github.com/VlachosGroup/pmutt/blob/master/docs/source/examples_jupyter/openmkm_io/inputs/NH3_Input_Data.xlsx) file. There are five sheets:
1. `refs` contains *ab-initio* and experimental data for a handful of gas species to calculate references
2. `species` contains *ab-initio* data for each specie
3. `beps` contains Bronsted-Evans-Polanyi relationships for reactions
4. `reactions` contains elementary steps
5. `lateral_interactions` contains lateral interactions between species
6. `phases` contains phases for the species

First, we change the working directory to the location of the Jupyter notebook.

In [1]:
import os
from pathlib import Path

# Find the location of Jupyter notebook
# Note that normally Python scripts have a __file__ variable but Jupyter notebook doesn't.
# Using pathlib can overcome this limiation
try:
    notebook_path = os.path.dirname(__file__)
except NameError:
    notebook_path = Path().resolve()
    
os.chdir(notebook_path)
input_path = './inputs/NH3_Input_Data_new.xlsx'

Below is a helper function to print tables easily.

In [2]:
import pandas as pd
from IPython.display import display

def disp_data(io, sheet_name):
    data = pd.read_excel(io=io, sheet_name=sheet_name, skiprows=[1])
    data = data.fillna(' ')
    display(data)    

**References**

In [3]:
disp_data(io=input_path, sheet_name='refs')

,name,elements.N,elements.H,elements.Ru,T_ref,HoRT_ref,potentialenergy,symmetrynumber,statmech_model,atoms,vib_wavenumber,vib_wavenumber.1,vib_wavenumber.2,vib_wavenumber.3
0,N2,2,0,0,298.15,0.000000,-16.63,2,IdealGas,./N2/CONTCAR,2744,,,
1,NH3,1,3,0,298.15,-18.380253,-19.54,3,IdealGas,./NH3/CONTCAR,3534,3464,1765,1139
2,H2,0,2,0,298.15,0.000000,-6.77,2,IdealGas,./H2/CONTCAR,4342,,,
3,Ru,0,0,1,298.15,0.000000,,,Placeholder,,,,,


**Species**

In [4]:
disp_data(io=input_path, sheet_name='species')

,name,elements.N,elements.H,elements.Ru,phase,n_sites,statmech_model,symmetrynumber,atoms,potentialenergy,...,vib_wavenumber.2,vib_wavenumber.3,vib_wavenumber.4,vib_wavenumber.5,vib_wavenumber.6,vib_wavenumber.7,vib_wavenumber.8,vib_wavenumber.9,vib_wavenumber.10,vib_wavenumber.11
0,N2,2,,,gas,,IdealGas,2,./N2/CONTCAR,-16.63,...,,,,,,,,,,
1,NH3,1,3,,gas,,IdealGas,3,./NH3/CONTCAR,-19.54,...,1765,1139,,,,,,,,
2,H2,,2,,gas,,IdealGas,2,./H2/CONTCAR,-6.77,...,,,,,,,,,,
3,N2(T),2,,,terrace,1,Harmonic,,,-17.24,...,347.343,335.674,62.076,32.1794,,,,,,
4,N(T),1,,,terrace,1,Harmonic,,,-9.34,...,475.454,,,,,,,,,
5,H(T),,1,,terrace,1,Harmonic,,,-4,...,797.904,,,,,,,,,
6,NH3(T),1,3,,terrace,1,Harmonic,,,-20.43,...,3364.52,1583.52,1582.07,1124.22,570.212,567.221,333.09,122.859,83.8286,70.6251
7,NH2(T),1,2,,terrace,1,Harmonic,,,-16.59,...,1503.02,698.869,625.596,615.94,475.13,298.12,153.25,,,
8,NH(T),1,1,,terrace,1,Harmonic,,,-13.21,...,710.581,528.526,415.196,410.131,,,,,,
9,TS1_NH3(T),1,3,,,1,Harmonic,,,-19.24,...,1723.85,1487.95,959.151,888.946,594.089,428.431,227.032,206.047,142.136,


**BEPs**

In [5]:
disp_data(io=input_path, sheet_name='beps')

,name,slope,intercept,direction,notes
0,N-H,0.29,23.23,cleavage,Values taken from https://github.com/VlachosGr...
1,NH-H,0.52,19.78,cleavage,Values taken from https://github.com/VlachosGr...
2,NH2-H,0.71,23.69,cleavage,Values taken from https://github.com/VlachosGr...


**Reactions**

In [6]:
disp_data(io=input_path, sheet_name='reactions')

,reaction_str,is_adsorption,direction,id
0,H2 + 2RU(T) = 2H(T) + 2RU(B),True,,
1,N2 + RU(T) = N2(T) + RU(B),True,,
2,NH3 + RU(T) = NH3(T) + RU(B),True,,
3,NH3(T) + RU(T)= NH2-H = NH2(T) + H(T) + RU(B),False,cleavage,
4,NH2(T) + RU(T) = NH-H = NH(T) + H(T) + RU(B),False,cleavage,
5,NH(T) + RU(T) = N-H = N(T) + H(T) + RU(B),False,cleavage,
6,2N(T) + RU(B) = TS4_N2(T) = N2(T) + RU(T),False,,
7,H2 + 2RU(S) = 2H(S) + 2RU(B),True,,
8,N2 + RU(S) = N2(S) + RU(B),True,,
9,NH3 + RU(S) = NH3(S) + RU(B),True,,


**Lateral Interactions**

In [7]:
disp_data(io=input_path, sheet_name='lateral_interactions')

,name_i,name_j,list.intervals,list.slopes
0,N(T),N(T),0,-52.6
1,N(T),H(T),0,-17.7
2,H(T),N(T),0,-17.7
3,H(T),H(T),0,-3.0
4,NH2(T),N(T),0,-20.7
5,N(S),N(S),0,-52.6
6,N(S),H(S),0,-17.7
7,H(S),N(S),0,-17.7
8,H(S),H(S),0,-3.0
9,NH2(S),N(S),0,-20.7


**Phases**

In [8]:
disp_data(io=input_path, sheet_name='phases')

,name,phase_type,density,site_density,list.phases,list.phases.1,dict.initial_state.RU(T),dict.initial_state.RU(S),dict.initial_state.NH3,note
0,gas,IdealGas,,,,,,,1,
1,bulk,StoichSolid,12.4,,,,,,,Ru Metal
2,terrace,InteractingInterface,,2.1671e-09,gas,bulk,1,,,Surface: Ru0001(0T)
3,step,InteractingInterface,,4.4385e-10,gas,bulk,,1,,Surface: Ru0001(0S)


## Designate Units
First, we will designate the units to write the CTI file.

In [9]:
from pmutt.omkm.units import Units

units = Units(length='cm', quantity='mol', act_energy='kcal/mol', mass='g', energy='kcal/mol')

## Reading data
Before we can initialize our species, we need the references.

### Reading References (optional)
We will open the [input spreadsheet](https://github.com/VlachosGroup/pmutt/blob/master/docs/source/examples_jupyter/openmkm_io/inputs/NH3_Input_Data.xlsx) and read the `refs` sheet.

In [10]:
from pmutt.io.excel import read_excel
from pmutt.empirical.references import Reference, References

try:
    refs_data = read_excel(io=input_path, sheet_name='refs')
except:
    # If references are not used, skip this section
    refs = None
else:
    refs = [Reference(**ref_data) for ref_data in refs_data]
    refs = References(references=refs)

### Reading Species

In [11]:
from pmutt.empirical.nasa import Nasa

# Lower and higher temperatures
T_low = 298. # K
T_high = 800. # K

species_data = read_excel(io=input_path, sheet_name='species')
species = []
species_phases = {}
for ind_species_data in species_data:
    # Initialize NASA from statistical mechanical data
    ind_species = Nasa.from_model(T_low=T_low, T_high=T_high, references=refs,
                                  **ind_species_data)
    species.append(ind_species)

    # Group the species by phase for later use
    try:
        species_phases[ind_species.phase].append(ind_species)
    except KeyError:
        species_phases[ind_species.phase] = [ind_species]

### Adding species from other empirical sources (optional)

In [12]:
import numpy as np
from pmutt.empirical.shomate import Shomate

Ar = Shomate(name='Ar', elements={'Ar': 1}, phase='gas', T_low=298., T_high=6000.,
             a=np.array([20.78600, 2.825911e-7, -1.464191e-7, 1.092131e-8, -3.661371e-8, -6.19735, 179.999, 0.]))

species.append(Ar)
species_phases['gas'].append(Ar)

### Reading BEP (optional)

In [13]:
from pmutt.omkm.reaction import BEP

try:
    beps_data = read_excel(io=input_path, sheet_name='beps')
except:
    beps = None
    species_with_beps = species.copy()
else:
    beps = []
    for bep_data in beps_data:
        beps.append(BEP(**bep_data))

    # Combine species and BEPs to make reactions
    species_with_beps = species + beps

### Read reactions

In [14]:
from pmutt import pmutt_list_to_dict
from pmutt.omkm.reaction import SurfaceReaction

# Convert species to dictionary for easier reaction assignment
species_with_beps_dict = pmutt_list_to_dict(species_with_beps)
reactions_data = read_excel(io=input_path, sheet_name='reactions')
reactions = []
# Store information about phases for later retrieval
reaction_phases = {}
for reaction_data in reactions_data:
    reaction = SurfaceReaction.from_string(species=species_with_beps_dict,
                                           **reaction_data)
    reactions.append(reaction)
    # Assign phase information
    reaction_species = reaction.get_species(include_TS=True)
    for ind_species in reaction_species.values():
        try:
            phase = ind_species.phase
        except AttributeError:
            pass
        # Assign if key already exists
        if phase in reaction_phases:
            if reaction not in reaction_phases[phase]:
                reaction_phases[phase].append(reaction)
        else:
            reaction_phases[phase] = [reaction]

### Read lateral interactions (optional)

In [15]:
from pmutt.mixture.cov import PiecewiseCovEffect

try:
    interactions_data = read_excel(io=input_path, sheet_name='lateral_interactions')
except:
    # If no lateral interactions exist, skip this section
    interactions = None
else:
    interactions = []
    interaction_phases = {}
    for interaction_data in interactions_data:
        interaction = PiecewiseCovEffect(**interaction_data)
        interactions.append(interaction)

        # Assign phase information
        phase = species_with_beps_dict[interaction.name_i].phase
        # Assign if key already exists
        if phase in interaction_phases:
            if interaction not in interaction_phases[phase]:
                interaction_phases[phase].append(interaction)
        else:
            interaction_phases[phase] = [interaction]

### Reading Phases

In [16]:
from pmutt.omkm.phase import IdealGas, InteractingInterface, StoichSolid

phases_data = read_excel(io=input_path, sheet_name='phases')
phases = []
# Group data related to previously collected data
additional_fields = {'species': species_phases,
                     'reactions': reaction_phases,
                     'interactions': interaction_phases}
for phase_data in phases_data:
    # Pre-processing relevant data
    phase_name = phase_data['name']
    phase_type = phase_data.pop('phase_type')

    # Add additional fields to phase data if present
    for field, phase_dict in additional_fields.items():
        try:
            phase_data[field] = phase_dict[phase_name]
        except (NameError, KeyError):
            pass

    # Create the appropriate object
    if phase_type == 'IdealGas':
        # Special rule where reactions are only in the gas phase if
        # all species belong to the gas phase
        del_indices = []
        for i, reaction in enumerate(phase_data['reactions']):
            # Reaction will be deleted if any of the species are a different phase
            valid_rxn = True
            for ind_species in reaction.get_species(include_TS=False).values():
                try:
                    ind_species_phase = ind_species.phase
                except AttributeError:
                    valid_rxn = False
                else:
                    if ind_species_phase != phase_name:
                        valid_rxn = False
                # Record reaction index if not valid
                if not valid_rxn:
                    del_indices.append(i)
                    break
        # Delete reactions that do not qualify
        if len(del_indices) == len(phase_data['reactions']):
            phase_data.pop('reactions')
        else:
            for del_i in sorted(del_indices, reverse=True):
                del phase_data['reactions'][del_i]
        phase = IdealGas(**phase_data)
    elif phase_type == 'StoichSolid':
        phase = StoichSolid(**phase_data)
    elif phase_type == 'InteractingInterface':
        phase = InteractingInterface(**phase_data)
    phases.append(phase)

## Write CTI File

In [17]:
from pmutt.io.omkm import write_cti

cti_path = './outputs/thermo.cti'
use_motz_wise = True

write_cti(reactions=reactions, species=species, phases=phases, units=units,
          lateral_interactions=interactions, filename=cti_path,
          use_motz_wise=use_motz_wise)

If you would prefer to return the file as a string instead of writing it, omit the ``filename``.

In [18]:
print(write_cti(reactions=reactions, species=species, phases=phases, units=units,
                lateral_interactions=interactions, use_motz_wise=use_motz_wise))

# File generated by pMuTT (v 1.2.14) on 2019-12-05 16:55:39.859601
# See documentation for OpenMKM CTI file here:
# https://vlachosgroup.github.io/openmkm/input

#-------------------------------------------------------------------------------
# UNITS
#-------------------------------------------------------------------------------
units(length="cm", time="s", quantity="mol", energy="kcal/mol",
      act_energy="kcal/mol", pressure="bar", mass="g")

#-------------------------------------------------------------------------------
# PHASES
#-------------------------------------------------------------------------------
ideal_gas(name="gas",
          elements="H Ar N",
          species="N2 NH3 H2 Ar")

stoichiometric_solid(name="bulk",
                     elements="Ru",
                     species="RU(B)",
                     density=12.4,
                     note="Ru Metal")

interacting_interface(name="terrace",
                      elements="Ru H N",
                      species=

## Write YAML File

The YAML file specifying the reactor configuration can also be written using the ``write_yaml`` function. Note that if:
- ``units`` is not specified, float values are assumed to be in SI units
- ``units`` is specified, float values are consistent with ``unit``'s attributes
- you would like a quantity to have particular units, pass the value as a string with the units  (e.g. 10 cm3/s).

In [19]:
from pmutt.io.omkm import write_yaml

yaml_path = './outputs/cstr.yaml'

write_yaml(filename=yaml_path, reactor_type='cstr', mode='isothermal',
           V=1., T=900., P=1., cat_abyv=1500, end_time=50, flow_rate=1.,
           transient=True, stepping='logarithmic', init_step=1e-15, atol=1e-15,
           rtol=1e-10, output_format='csv', phases=phases, units=units)

If you would prefer to return the file as a string instead of writing it, omit the ``filename``.

In [20]:
print(write_yaml(reactor_type='cstr', mode='isothermal', V=1., T=900., P=1., cat_abyv=1500,
                 end_time=50, flow_rate=1., transient=True, stepping='logarithmic',
                 init_step=1e-15, atol=1e-15, rtol=1e-10, output_format='csv', phases=phases,
                 units=units))

# File generated by pMuTT (v 1.2.14) on 2019-12-05 16:55:39.951601
# See documentation for OpenMKM YAML file here:
# https://vlachosgroup.github.io/openmkm/input
inlet_gas:
    flow_rate: "1.0 cm3/s"
phases:
    bulk:
        name: bulk
    gas:
        initial_state: "NH3:1.0"
        name: gas
    surfaces:
    -   initial_state: "RU(T):1.0"
        name: terrace
    -   initial_state: "RU(S):1.0"
        name: step
reactor:
    cat_abyv: "1500 /cm"
    mode: "isothermal"
    pressure: "1.0 bar"
    temperature: 900.0
    type: "cstr"
    volume: "1.0 cm3"
simulation:
    end_time: "50 s"
    init_step: 1.0e-15
    output_format: "csv"
    solver:
        atol: 1.0e-15
        rtol: 1.0e-10
    stepping: "logarithmic"
    transient: true

